In [1]:
import torch 
from torch import nn

import ray
from ray.rllib.agents import ppo
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.utils.annotations import override

#from models import VisualEncoder
from train import *
from wrappers_2 import *

/root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
class VisualEncoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=2, stride=2, padding=0),  
            nn.ELU(),
            nn.Conv2d(32, 32, kernel_size=2, stride=2, padding=0), 
            nn.ELU(),
            nn.Conv2d(32, 64, kernel_size=2, stride=2, padding=0), 
            nn.ELU(),
            nn.Conv2d(64, 128, kernel_size=2, stride=2, padding=0),
            nn.ELU(), 
            nn.Conv2d(128, 256, kernel_size=2, stride=2, padding=0),
            nn.ELU(),
            nn.Conv2d(256, 512, kernel_size=2, stride=2, padding=0),
            nn.ELU(),
            nn.Flatten(),
        )

    def forward(self, x):
        return self.cnn(x)

In [3]:
from torch.nn.functional import one_hot

class MyModelClass(TorchModelV2, nn.Module):
    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs, model_config, name)
        nn.Module.__init__(self)
        visual_features_dim = 512
        target_features_dim = 9 * 11 * 11 
        self.visual_encoder = VisualEncoder()
        self.visual_encoder.load_state_dict(
            torch.load("/IGLU-Minecraft/models/AngelaCNN/encoder_weigths.pth", map_location=torch.device('cpu'))
        )
        self.target_encoder = nn.Sequential(
            nn.Conv3d(7, 1, kernel_size=1, stride=1, padding=0),
            nn.ELU(),
        )
        policy_hidden_dim = 256 
        self.policy_network = nn.Sequential(
            nn.Linear(visual_features_dim + target_features_dim, 1024),
            nn.ELU(),
            nn.Linear(1024, 512),
            nn.ELU(),
            nn.Linear(512, policy_hidden_dim),
            nn.ELU(),
            nn.Linear(policy_hidden_dim, policy_hidden_dim),
            nn.ELU(),
            #nn.Linear(policy_hidden_dim, policy_hidden_dim),
            #nn.ELU(),
        )
        self.action_head = nn.Linear(policy_hidden_dim, action_space.n)
        self.value_head = nn.Linear(policy_hidden_dim, 1)
        self.last_value = None
        
        self.use_cuda = torch.cuda.is_available()
        if self.use_cuda:
            self.visual_encoder.cuda()
            self.target_encoder.cuda()
            self.policy_network.cuda()
            self.action_head.cuda()
            self.value_head.cuda()
        
    @override(TorchModelV2)
    def forward(self, input_dict, state, seq_lens):
        obs = input_dict['obs']
        pov = obs['pov'].permute(0, 3, 1, 2).float() / 255.0
        target = one_hot(obs['target_grid'].long(), num_classes=7).permute(0, 4, 1, 2, 3).float()
        if self.use_cuda:
            pov.cuda()
            target.cuda()
            
        with torch.no_grad():
            visual_features = self.visual_encoder(pov)
            
        target_features = self.target_encoder(target)
        target_features = target_features.reshape(target_features.shape[0], -1)
        features = torch.cat([visual_features, target_features], dim=1)
        features = self.policy_network(features)
        action = self.action_head(features)
        self.last_value = self.value_head(features).squeeze(1)
        return action, state
    
    @override(TorchModelV2)
    def value_function(self):
        assert self.last_value is not None, "must call forward() first"
        return self.last_value

In [4]:
ModelCatalog.register_custom_model("my_torch_model", MyModelClass)

In [5]:
class VisualObservationWrapper(ObsWrapper):
    def __init__(self, env, include_target=False):
        super().__init__(env)
        self.observation_space = {   
            'pov': gym.spaces.Box(low=0, high=255, shape=(64, 64, 3)),
            'inventory': gym.spaces.Box(low=0.0, high=20.0, shape=(6,)),
            'compass': gym.spaces.Box(low=-180.0, high=180.0, shape=(1,))
        }
        if include_target:
            self.observation_space['target_grid'] = \
                gym.spaces.Box(low=0, high=6, shape=(9, 11, 11))
        self.observation_space = gym.spaces.Dict(self.observation_space)

    def observation(self, obs, reward=None, done=None, info=None):
        if info is not None:
            if 'target_grid' in info:
                target_grid = info['target_grid']
                del info['target_grid']
            else:
                logger.error(f'info: {info}')
                if hasattr(self.unwrapped, 'should_reset'):
                    self.unwrapped.should_reset(True)
                target_grid = self.env.unwrapped.tasks.current.target_grid
        else:
            target_grid = self.env.unwrapped.tasks.current.target_grid
        return {
            'pov': obs['pov'].astype(np.float32),
            'inventory': obs['inventory'],
            'compass': np.array([obs['compass']['angle'].item()]),
            'target_grid': target_grid
        }

In [6]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

class RewardWrapper(gym.RewardWrapper):
    def __init__(self, env):
        super().__init__(env)
    
    def reward(self, rew):
        if rew == 0:
            rew = -0.01
        if abs(rew) == 1:
            rew /= 10
        return rew
    
def env_creator(env_config):
    env = gym.make('IGLUSilentBuilder-v0', max_steps=250)
    env.update_taskset(TaskSet(preset=['C12']))
    #env = PovOnlyWrapper(env)
    env = VisualObservationWrapper(env, include_target=True)
    env = SelectAndPlace(env)
    env = Discretization(env, flat_action_space('human-level'))
    env = RewardWrapper(env)
    return env

from ray.tune.registry import register_env
register_env("my_env", env_creator)

from ray import tune
from ray.rllib.agents.ppo import PPOTrainer

In [ ]:
from ray.tune.integration.wandb import WandbLogger

analysis = tune.run(PPOTrainer, 
         config={
             "env": "my_env", 
             "framework": "torch",
             "num_gpus": 1,
             "num_workers": 3,
             "sgd_minibatch_size": 256,
             "clip_param": 0.2,
             "entropy_coeff": 0.01,
             "lambda": 0.95,
             "train_batch_size": 1000,
             #"gamma": 0.99,
             "model": {
                    # Specify our custom model from above.
                    "custom_model": "my_torch_model",
                    # Extra kwargs to be passed to your model's c'tor.
                    "custom_model_config": {},
              },
             "logger_config": {
                  "wandb": {
                      "project": "IGLU-Minecraft",
                      "name": "PPO MultiTask (C12) pretrained (AngelaCNN) (3 noops after placement) r: -0.01 div10"
                  }
              }

        },
        loggers=[WandbLogger],
        local_dir="/IGLU-Minecraft/checkpoints/C12",
        keep_checkpoints_num=50,
        checkpoint_freq=5,
        checkpoint_at_end=True,
        #restore="/IGLU-Minecraft/checkpoints/4_tasks/PPO_2021-11-08_20-28-45/PPO_my_env_78cf0_00000_0_2021-11-08_20-28-45/checkpoint_000050/checkpoint-50"
        )

/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/_private/services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)


Trial name,status,loc
PPO_my_env_c1ec9_00000,PENDING,


2021-11-13 21:29:18,085	INFO wandb.py:170 -- Already logged into W&B.
(pid=97382) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=97382)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
2021-11-13 21:29:19,713	ERROR syncer.py:72 -- Log sync requires rsync to be installed.
wandb: Currently logged in as: linar (use `wandb login --relogin` to force relogin)
(pid=97382) 2021-11-13 21:29:21,589	WARNING ppo.py:143 -- `train_batch_size` (1000) cannot be achieved with your other settings (num_workers=3 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 333.
(pid=97382) 2021-11-13 21:29:21,589	INFO ppo.py:159 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(pid=97382) 2021-11-13 21:29:21,589	INFO trainer.py:728 -- Current log

(pid=97370) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=97370)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=97370) /root/miniconda/envs/py37/lib/python3.7/runpy.py:125: RuntimeWarning: 'minerl_patched.utils.process_watcher' found in sys.modules after import of package 'minerl_patched.utils', but prior to execution of 'minerl_patched.utils.process_watcher'; this may result in unpredictable behaviour
(pid=97370)   warn(RuntimeWarning(msg))
(pid=97375) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=97375)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=97375) /root/miniconda/envs/py37/lib/python3.7/runpy.py:125: RuntimeWarning: 'minerl_patched.utils.process_watcher' found in sys.modules after import of package 'minerl_patched.utils', but pri

Result for PPO_my_env_c1ec9_00000:
  agent_timesteps_total: 1998
  custom_metrics: {}
  date: 2021-11-13_21-31-01
  done: false
  episode_len_mean: 101.22222222222223
  episode_media: {}
  episode_reward_max: 2.82
  episode_reward_mean: -0.6011111111111115
  episode_reward_min: -1.1400000000000008
  episodes_this_iter: 18
  episodes_total: 18
  experiment_id: 3471556cf6b5442e8ddf461993f1dbed
  hostname: linar-Z390-GAMING-X
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.2
          cur_lr: 5.000000000000001e-05
          entropy: 2.882840056646438
          entropy_coeff: 0.009999999999999998
          kl: 0.005703404819647301
          policy_loss: 0.16548543302785781
          total_loss: 0.1672038088951792
          vf_explained_var: -0.32715457677841187
          vf_loss: 0.02940609449565056
    num_agent_steps_sampled: 1998
    num_agent_steps_trained: 1998
    num_steps_sampled: 1998
    num_steps_trained: 1998
  iterations_since_restor

...

Result for PPO_my_env_c1ec9_00000:
  agent_timesteps_total: 507492
  custom_metrics: {}
  date: 2021-11-13_23-19-59
  done: false
  episode_len_mean: 97.88
  episode_media: {}
  episode_reward_max: 21.07999999999994
  episode_reward_mean: 20.42709999999992
  episode_reward_min: -0.060000000000000005
  episodes_this_iter: 22
  episodes_total: 5004
  experiment_id: 3471556cf6b5442e8ddf461993f1dbed
  hostname: linar-Z390-GAMING-X
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.253125
          cur_lr: 5.000000000000001e-05
          entropy: 1.3304259964397975
          entropy_coeff: 0.009999999999999998
          kl: 0.006856749182977698
          policy_loss: 4.055843289409365e-05
          total_loss: 0.7725613987173087
          vf_explained_var: 0.9780187010765076
          vf_loss: 0.7840895112958692
    num_agent_steps_sampled: 507492
    num_agent_steps_trained: 507492
    num_steps_sampled: 507492
    num_steps_trained: 507492
  iterat

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_c1ec9_00000,RUNNING,192.168.3.5:97382,254,6623.43,507492,20.4271,21.08,-0.06,97.88


(pid=97370) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=97370)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Result for PPO_my_env_c1ec9_00000:
  agent_timesteps_total: 509490
  custom_metrics: {}
  date: 2021-11-13_23-20-37
  done: false
  episode_len_mean: 97.26
  episode_media: {}
  episode_reward_max: 21.07999999999994
  episode_reward_mean: 20.21819999999992
  episode_reward_min: -0.060000000000000005
  episodes_this_iter: 20
  episodes_total: 5024
  experiment_id: 3471556cf6b5442e8ddf461993f1dbed
  hostname: linar-Z390-GAMING-X
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.253125
          cur_lr: 5.000000000000001e-05
          entropy: 1.3472122016407195
          entropy_coeff: 0.009999999999999998
          kl: 0.004030742534568872
          policy_loss: 0.00155059899247828
          total_loss: 0.6336043970304586
          vf_explained_var: 0.981549084186554
          vf_loss: 0.6445056505856059
    num_agent_steps_sampled: 509490
    num_agent_steps_trained: 509490
    num_steps_sampled: 509490
    num_steps_trained: 509490
  iteration

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_c1ec9_00000,RUNNING,192.168.3.5:97382,255,6661.89,509490,20.2182,21.08,-0.06,97.26
